# VacationPy
----

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json
import requests

# Import API key
from api_keys import g_key
from api_keys import weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# read the csv into a dataframe
cities_df = pd.read_csv('./citiesA.csv')
# drop any na rows
cities_df.dropna()
# Check the dataframe by examining just the head
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hong Kong,40,HK,1584110208,78,22.29,114.16,77.00,5.82
1,1,Nikolskoye,90,RU,1584110885,74,59.70,30.79,37.40,17.90
2,2,Bluff,11,NZ,1584110859,96,-46.60,168.33,45.23,3.60
3,3,Payo,100,ID,1584110887,94,-3.75,103.64,68.99,3.71
4,4,Saint-Philippe,100,RE,1584110889,74,-21.36,55.77,81.00,21.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
# Store humidity information in humidity
humidity = cities_df["Humidity"].astype(float)

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
coordinates = cities_df[["Lat", "Lng"]]

In [26]:
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout,center=(0.0, 0.0), zoom_level=1.8)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius=1)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(coordinates, weights=humidity))

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# A max temperature lower than 80 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
hotel_df =   cities_df[(cities_df['Max Temp'] >= 60) 
          & (cities_df['Max Temp'] < 80) 
          & (cities_df['Wind Speed'] < 10)
          & (cities_df['Cloudiness'] == 0)].reset_index()

# set all the values to '' for hotel name
hotel_df.insert(loc=1, column='Hotel Name', value=['' for i in range(hotel_df.shape[0])])
#hotel_df

,index,Hotel Name,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,26,,26,Wanning,0,CN,1584111055,92,18.80,110.40,75.47,6.60
1,42,,42,Zygos,0,GR,1584111058,63,41.02,24.38,73.99,5.82
2,55,,55,Mount Isa,0,AU,1584110921,36,-20.73,139.50,78.80,5.82
3,58,,58,Chiang Rai,0,TH,1584111061,65,19.91,99.83,77.00,2.24
4,73,,73,Hermanus,0,ZA,1584110869,74,-34.42,19.23,78.01,3.00
5,147,,147,Teacapan,0,MX,1584111077,56,22.55,-105.75,76.73,1.83
6,170,,170,Finca Blanco,0,PA,1584111081,51,8.38,-82.87,72.00,4.00
7,253,,253,Dingle,0,PH,1584111095,76,11.00,122.67,74.39,8.21
8,339,,339,Chittarkonda,0,IN,1584111111,49,18.12,82.08,75.11,5.10
9,346,,346,Dulag,0,PH,1584111112,80,10.95,125.03,76.55,4.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each row finding the nearest hotel by lat,lng coordinates
# and populate the 'Hotel Name' column with the closest hotel name
# The hotel names are also printed to the screen

for index, row in hotel_df.iterrows():
    # put the lat,lng values into string location
    # so it can help build the query
    location = f"{row['Lat']}, {row['Lng']}"
    # set up parameters
    params = {
        "location": location,  # city coords,
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
    }
    
    # print(params)   # for debugging    
    # make the request
    response = requests.get(base_url, params=params).json()
   
    # extract results
    results = response['results']
       
    try:
        print(f"The closest hotel to the city of {row['City']} is the {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = {results[0]['name']}      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------------------------------------------------------")
    
#hotel_df['Hotel Name']
#hotel_df

The closest Hotel to the city of Wanning is the People Hotel.
------------------------------------------------------------
The closest Hotel to the city of Zygos is the Villa in Village.
------------------------------------------------------------
The closest Hotel to the city of Mount Isa is the Aboriginal Hostels Ltd..
------------------------------------------------------------
The closest Hotel to the city of Chiang Rai is the กิมล้วน.
------------------------------------------------------------
The closest Hotel to the city of Hermanus is the Hermanus Bed and Breakfast.
------------------------------------------------------------
The closest Hotel to the city of Teacapan is the Lorenzo's South.
------------------------------------------------------------
The closest Hotel to the city of Finca Blanco is the Parque Infantil de Finca Blanco.
------------------------------------------------------------
The closest Hotel to the city of Dingle is the Camp Pasica.
-----------------------

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#locations  # for debugging
    

In [23]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_df['Hotel Name'])

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### End of notebook